## 전이학습(Transfer Learning)

- VGG
- Inception
- ResNet

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from torch import nn, optim

import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [ ]:
%cd /content/gdrive/My Drive/pytorch/

/content/gdrive/My Drive/pytorch


In [ ]:
#!wget https://github.com/lucidfrontier45/PyTorch-Book/raw/master/data/taco_and_burrito.tar.gz
#!tar -zxvf taco_and_burrito.tar.gz

In [ ]:
tr_imgs = ImageFolder('./taco_and_burrito/train', transform = transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor()]))
te_imgs = ImageFolder('./taco_and_burrito/test', transform = transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor()]))

In [ ]:
train_loader = DataLoader(tr_imgs, batch_size = 32, shuffle = True)
test_loader = DataLoader(te_imgs, batch_size = 32, shuffle = False)

In [ ]:
print(tr_imgs.classes)
print(tr_imgs.class_to_idx)

['burrito', 'taco']
{'burrito': 0, 'taco': 1}


In [ ]:
from torchvision.models import resnet18

In [ ]:
net = resnet18(pretrained = True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
for param in net.parameters() :
  param.requires_grad = False

In [ ]:
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

In [ ]:
def eval_net(net, data_loader, device = 'cpu') :

  net.eval()

  ys = []
  y_preds = []

  for x, y, in data_loader :
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad() :
      _, y_pred = net(x).max(1)

    ys.append(y)
    y_preds.append(y_pred)

  ys = torch.cat(ys)
  y_preds = torch.cat(y_preds)

  acc = (ys == y_preds).float().sum() / len(ys)

  return acc.item()

In [ ]:
def train_net(net, train_loader, test_loader, only_fc = True, optimizer_cls = optim.Adam, loss_fn = nn.CrossEntropyLoss(), n_iter = 5, device = 'cpu') :

  tr_losses = []
  tr_acc = []
  val_acc = []

  if only_fc :
    optimizer = optimizer_cls(net.fc.parameters())
  else :
    optimizer = optimizer_cls(net.parameters())

  for epoch in range(n_iter) :
    running_loss = 0.0

    net.train()
    n = 0
    n_acc = 0

    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)) :

      xx = xx.to(device)
      yy = yy.to(device)

      h = net(xx)

      loss = loss_fn(h, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

      n += len(xx)

      _, y_pred = h.max(1)

      n_acc += (yy == y_pred).float().sum().item()

    tr_losses.append(running_loss / i)
    tr_acc.append(n_acc / n)

    val_acc.append(eval_net(net, test_loader, device))

    print(f"Epoch : {epoch}  / train_loss : {tr_losses[-1]} / train_acc : {tr_acc[-1]} / validation_acc : {val_acc[-1]}")

In [ ]:
net.to("cuda:0")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
train_net(net, train_loader, test_loader, n_iter = 5, device = 'cuda:0')

100%|██████████| 23/23 [00:03<00:00,  6.61it/s]


Epoch : 0  / train_loss : 0.34110679951581085 / train_acc : 0.8623595505617978 / validation_acc : 0.7666667103767395


100%|██████████| 23/23 [00:03<00:00,  6.72it/s]


Epoch : 1  / train_loss : 0.3340230861848051 / train_acc : 0.8764044943820225 / validation_acc : 0.8333333730697632


100%|██████████| 23/23 [00:03<00:00,  6.61it/s]


Epoch : 2  / train_loss : 0.3646178814497861 / train_acc : 0.8398876404494382 / validation_acc : 0.8833333849906921


100%|██████████| 23/23 [00:03<00:00,  6.74it/s]


Epoch : 3  / train_loss : 0.30580303276127035 / train_acc : 0.8820224719101124 / validation_acc : 0.7833333611488342


100%|██████████| 23/23 [00:03<00:00,  6.70it/s]


Epoch : 4  / train_loss : 0.3004496307535605 / train_acc : 0.8834269662921348 / validation_acc : 0.8833333849906921


FC 레이어 외에는 파라미터가 고정되어 있어 매회 똑같은 계산이 불필요하게 발생한다.

따라서 더미 레이어(입력 -> 출력)를 만들어 FC를 변경하는 것이 범용성이 높다.

In [ ]:
class FlattenLayer(nn.Module) :

  def forward(self, x) :
    sizes = x.size()
    return x.view(sizes[0], -1)

In [ ]:
class IdentityLayer(nn.Module) :

  def forward(self, x) :
    return x

In [ ]:
net = resnet18(pretrained = True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for p in net.parameters() :
  p.requires_grad = False

In [ ]:
net.fc = IdentityLayer()